# Imports

The required packages to run this notebook.  
If any are missing use ``!{sys.executable} -m pip install <package>`` to install the package in the notebook environment

In [1]:
import os.path
import gc 
import pandas as pd
import csv
import sys
from ctypes import cdll, CDLL
from urllib import parse
#!{sys.executable} -m pip install beautifulsoup4
from bs4 import BeautifulSoup
#!{sys.executable} -m pip install nbimporter
import nbimporter
import ARCH_Helper as ah

# Download an archive
The script below makes a data folder with a subfolder for the dataset. Dataset option are "World", "German" or "Dutch", select these by altering the ``dataset`` variable. The script wil skip files that are already present on the system. If you want to redownload the files first remove the old files manually.  
If the download fails due to connection errors, rerun the script and will will notice that he file did not decompress and try again.  
Note: The html-file-information.csv can be very large and take several hours and fail. Consider putting the code below code in a loop if you want to run it over night, if the download fails it will try again and if successfull it will say it already has the desired file.

In [2]:
# Dataset option are "World", "German" or "Dutch"
Dataset = "World"

!mkdir data
!mkdir data/{Dataset}

if Dataset == "German":
    if not os.path.exists("data/German/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainFrequencyExtraction/domain-frequency.csv.gz?access=VRQ4COI5RFEB6XTJZNQTBRLEZTTHJERL" --output data/German/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/German/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainGraphExtraction/domain-graph.csv.gz?access=JKEPGQ6MUC72JQB23IXOC4KOLGJYDSMN" --output data/German/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/German/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/css-file-information.csv.gz?access=I2WP4REJA3NOBU3TCAAL3OIGJKNXM46R" --output data/German/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/German/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/js-file-information.csv.gz?access=M3QSMFPLEHPZPWZIFSMZ6CT2OO7WYQ4M" --output data/German/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
        
    if not os.path.exists("data/German/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-germanNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/html-file-information.csv.gz?access=E3BUHKL5P4Q3TD4LGXZV2AOMERYQ3GWL" --output data/German/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")
        
if Dataset == "World":
    if not os.path.exists("data/World/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainFrequencyExtraction/domain-frequency.csv.gz?access=SMSQY3G6IGKGRWVLGCWMA7DMCHBCKQ4K" --output data/World/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/World/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/DomainGraphExtraction/domain-graph.csv.gz?sample=true&access=BZTA7LW5LUNMPGKWQMLKAUAXBV2E2AEC" --output data/World/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/World/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/css-file-information.csv.gz?access=UZ4PWVRXXWPF53BHM7TTYNK24P7YAIXQ" --output data/World/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/World/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/js-file-information.csv.gz?access=54X7IV7QOOAJWGHKPRGDI7HIR6W6GKPI" --output data/World/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
    
    if not os.path.exists("data/World/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-worldNews_1st-in-month_1-hop_unique-EXTRACTION-20210818232425/TextFilesInformationExtraction/html-file-information.csv.gz?access=JQEDT3PRXOS6OXZ7LA5EC55HQJVUASDX" --output data/World/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")

if Dataset == "Dutch":
    if not os.path.exists("data/Dutch/domain-frequency.csv"):
        print("domain-frequency.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/DomainFrequencyExtraction/domain-frequency.csv.gz?access=QUQTRHVNIDKXN62S4D62XFHCFI7VDSH7" --output data/Dutch/domain-frequency.csv.gz
        print("domain-frequency.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/Dutch/domain-graph.csv"):
        print("domain-graph.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/DomainGraphExtraction/domain-graph.csv.gz?access=JUNESOMDTBYDNCCGVJI5MJHW45KHAIJZ" --output data/Dutch/domain-graph.csv.gz
        print("domain-graph.csv has been downloaded")
    else:
        print("domain-frequency.csv already exists")

    if not os.path.exists("data/Dutch/css-file-information.csv"):
        print("css-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/css-file-information.csv.gz?access=IGBKSGXJBL3L4IJ2LXS5NPJTOY77JCT5" --output data/Dutch/css-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("css-file-information.csv already exists")

    if not os.path.exists("data/Dutch/js-file-information.csv"):
        print("js-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/js-file-information.csv.gz?access=RPZKGWUTWJBIRKOKKHGECB7W4OHUATMO" --output data/Dutch/js-file-information.csv.gz
        print("css-file-information.csv has been downloaded")
    else:
        print("js-file-information.csv already exists")
    
    if not os.path.exists("data/Dutch/html-file-information.csv"):
        print("html-file-information.csv is not present")
        !curl "https://webdata.archive-it.org/ait/arch:cohort.helmond/research_services/download/SPECIAL-dutchNews_1st-in-month_1-hop_unique-EXTRACTION-20210916172606/TextFilesInformationExtraction/html-file-information.csv.gz?access=52ZOSNQQMFWKW42WMQPNMGKYLEVUQHUC" --output data/Dutch/html-file-information.csv.gz
        print("html-file-information.csv has been downloaded")
    else:
        print("html-file-information.csv already exists")

unzip = "find data/" +Dataset +" -name '*.gz' -exec gunzip {} \;"
!{unzip}

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/World’: File exists
domain-frequency.csv already exists
domain-frequency.csv already exists
css-file-information.csv already exists
js-file-information.csv already exists
html-file-information.csv already exists


# Extract wepages holding disqus.com/embed.js in their HTML

The code bellow looks through a dataset for any mention of disqus.js, an indicator of the presence of the disqus commenting system. The input for ```filter_csv_content_regex``` is the desired dataset file, the regular expression and the chunksize.

In [3]:
# Options are "data/German/html-file-information.csv" "data/World/html-file-information.csv" and "data/Dutch/html-file-information.csv"
# or any other dataset on the system.
disqus_df = ah.filter_csv_content_regex("data/World/html-file-information.csv", "(?i)disqus\.com\/embed\.js", 10000)

Count the pages with disqus.js in their HTML

In [4]:
len(disqus_df)

6840

Note that this finds more hits than grep. It seems the csv file is too large for grep. the same query in awk does give the expected number of results

```grep -i -n 'disqus\.js' C/Users/rjans/Desktop/ARCH/Data/disqus.csv```  
vs  
```awk -e '/disqus\.js/ {print $0}' C/Users/rjans/Desktop/ARCH/Data/html-file-information-German.csv```

Show the unique domains

In [5]:
print(disqus_df['url'].apply(ah.to_domain).unique())

['aljazeera.com' 'blogs.aljazeera.com' 'men.24.com' 'blogs.abcnews.com'
 'bloomberg.com' 'popwatch.ew.com' 'cnn.com' 'abcnews.go.com'
 'liveshots.blogs.foxnews.com' 'magazine.foxnews.com'
 'gretawire.foxnewsinsider.com' 'csmonitor.com' 'photos.essence.com'
 'myfoxchicago.com' 'myfoxla.com' 'myfoxmemphis.com' 'myfoxorlando.com'
 'mercurynews.com' 'spokesman.com' 'sfist.com' 'blogs.sfweekly.com'
 'tpmmuckraker.talkingpointsmemo.com' 'topgear.com' 'mirror.co.uk'
 'fashion.telegraph.co.uk' 'mweb.co.za' 'sagoodnews.co.za'
 'independent.co.uk' 'telegraph.co.uk' 'rawstory.com' 'pcmag.com'
 'politico.com' 'voices.news24.com' 'newser.com' 'newsmax.com'
 'english.aljazeera.net' 'lazygamer.net' 'washingtontimes.com'
 'communities.washingtontimes.com' 'propublica.org' 'scpr.org']


Save the results in a seperate CSV (only needs to be done once)

In [6]:
#disqus_df.to_csv("disqus.csv")

Convert **content** column to single line.

In [7]:
for line in disqus_df.content:
    disqus_df.content = disqus_df.content.replace(line, " ".join(line.splitlines()))

Save as single line content in a seperate CSV (only needs to be done once)

In [8]:
#disqus_df.to_csv("disqus_single_line.csv")

# For the found pages extract the comment structure

Extract the disqus comments from the div tag with ```id="disqus_thread"```

In [9]:
comments = []
for line in disqus_df.content:
    soup = BeautifulSoup(line, 'html.parser')
    try:
        comments.append(soup.find('div', attrs={'id': 'disqus_thread'}).prettify()) # .prettify() makes the HTML of the comments more human readable
    except AttributeError:
        comments.append("")
        
disqus_df['comments'] = comments

In [10]:
#pd.set_option('display.max_colwidth', 500) 
disqus_df[['crawl_date','url','comments']]

,crawl_date,url,comments
0,20140502,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
1,20140501,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
2,20140502,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
3,20140531,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
4,20150102,http://www.aljazeera.com/news/middleeast/2014/...,"<div id=""disqus_thread"">\n</div>\n"
...,...,...,...
6835,20120105,http://www.independent.co.uk/environment/natur...,"<div id=""disqus_thread"">\n</div>\n"
6836,20110301,http://www.independent.co.uk/environment/scien...,"<div id=""disqus_thread"">\n <div class=""clearfi..."
6837,20120105,http://www.independent.co.uk/environment/the-1...,"<div id=""disqus_thread"">\n</div>\n"
6838,20130502,http://www.independent.co.uk/extras/indybest/o...,"<div id=""disqus_thread"">\n</div>\n"


# Add an internet archive URL to the dataframe

In [11]:
ah.add_ia_url(disqus_df)

Filter out comment fields that are so short that they do net contain comments (based on the length).  
32 characters is currently the shortest 

In [12]:
#pd.set_option('display.max_colwidth', 50)
disqus_df[disqus_df['comments'].apply(lambda x: len(x)>32)][['comments','IA_url']]

,comments,IA_url
235,"<div class=""loading"" id=""disqus_thread"">\n</di...",https://web.archive.org/web/20150522/http://bl...
240,"<div class=""loading"" id=""disqus_thread"">\n</di...",https://web.archive.org/web/20150511/http://bl...
248,"<div id=""disqus_thread"">\n <script>\n var dis...",https://web.archive.org/web/20150127/http://ww...
249,"<div id=""disqus_thread"">\n <script>\n var dis...",https://web.archive.org/web/20140301/http://ww...
734,"<div class=""loading"" id=""disqus_thread"">\n</di...",https://web.archive.org/web/20150505/http://ab...
...,...,...
5529,"<div id=""disqus_thread"">\n <div class=""clearfi...",https://web.archive.org/web/20110412/http://ww...
5570,"<div class="""" id=""disqus_thread"">\n</div>\n",https://web.archive.org/web/20130304/http://ww...
6825,"<div id=""disqus_thread"">\n <div class=""clearfi...",https://web.archive.org/web/20110408/http://ww...
6828,"<div id=""disqus_thread"">\n <div class=""clearfi...",https://web.archive.org/web/20110927/http://ww...


Display a specific comment

In [13]:
ah.display_comment(disqus_df, 0)

<div id="disqus_thread">
</div>



# Comparing dataframes

Creates a second dataframe with a different regex query 

In [14]:
#disqus_df_2 = ah.filter_csv_content_regex("data/World/html-file-information.csv", "(?i)disqus\.js", 10000)

Show the overlap of two dataframes

In [15]:
#ah.dataframe_intersection(disqus_df[['crawl_date','url']],disqus_df_2[['crawl_date','url']])

Show the difference between the two datasets. So only shows results that are in one dataset but not in both.

In [16]:
#ah.dataframe_difference(disqus_df[['crawl_date','url']],disqus_df_2[['crawl_date','url']])

# Testing scratchbook

Reloads the ARCH Helper module after update.  
Normally this would be done with autoreload Ipython magic, but for some reason it does not work. 

In [1]:
import importlib
importlib.reload(ah)
import ARCH_Helper as ah

NameError: name 'ah' is not defined

In [20]:
diff = ah.dataframe_difference(disqus_df,ah.filter_df_content_regex(disqus_df, "(?i)disqus_thread"))
